# Materials API - Exercise 2: Using the MPRester and Pymatgen to Find Materials With Exotic Mechanical Properties

The tetragonal SiO$_2$ polymorph $\alpha$-cristobalite is one of the very few crystalline materials known to have a negative average Poisson's ratio, which means that its cross-section expands under tensile strain rather than contracting. This property can be extremely useful in a variety of applications such as scratch-resistant coatings and high-toughness ceramics. 

Why does $\alpha$-cristobalite exhibit this property while other materials do not? The prevailing hypothesis is that $\alpha$-cristobalite's negative Poisson's ratio is a result of its crystal structure. If that's the case, then perhaps we can find other materials with this exotic property by looking for materials with similar structures and then calculating their Poisson's ratios.

## Step 1: Retrieve the structure of $\alpha$-cristobalite

First, we should find the ground state structure for $\alpha$-cristobalite, which has the spacegroup $P4_12_12$ [92]. Remember, stable materials have *low* energy above hull (`energy_above_hull`).

_Hint: Query using `chemsys_formula` and `spacegroup_number` for the desired compound and then select the one with the lowest `energy_above_hull` if there are multiple results. Consider using the `sort_field` and `ascending` input parameters of the query method._

In [26]:
# Your code here
from mp_api.matproj import MPRester

with MPRester("<your API key>") as mpr:
    ac = mpr.query(spacegroup_number=92, chemsys_formula="SiO2", sort_field="energy_above_hull", ascending=True, fields=["material_id", "structure", "energy_above_hull"])


In [29]:
print([entry.energy_above_hull for entry in ac])

[0.0029292908333324874, 0.16456397777777632, 0.16702923694444216, 0.18101650749999898]


In [30]:
ac_structure = ac[0].structure
ac_mpid = ac[0].material_id

In [31]:
print(ac_mpid, ac_structure)

mp-6945 Full Formula (Si4 O8)
Reduced Formula: SiO2
abc   :   5.084701   5.084701   7.098579
angles:  90.000000  90.000000  90.000000
Sites (12)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Si    0.70555   0.70555   0.5
  1  Si    0.79445   0.20555   0.75
  2  Si    0.20555   0.79445   0.25
  3  Si    0.29445   0.29445   0
  4  O     0.905897  0.758978  0.325641
  5  O     0.094103  0.241022  0.825641
  6  O     0.758978  0.905897  0.674359
  7  O     0.258978  0.594103  0.075641
  8  O     0.241022  0.094103  0.174359
  9  O     0.405897  0.741022  0.424359
 10  O     0.594103  0.258978  0.924359
 11  O     0.741022  0.405897  0.575641


## Step 2: Build a structure comparison engine and test it out

The code below creates a structure matcher object that can be used to compare if two structures are simlar (loose tolerances = similar, tight tolerances = identical). Please verify that the structure matcher works for identical structure by comparing the structure of AC with itself. 

_Hint: Replace <YOUR CODE HERE> with the two structures you want to compare, separated by a comma._

In [32]:
##### Don't edit code below #####

from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.analysis.structure_matcher import FrameworkComparator
# This line initializes our comparator. Please don't play with its tolerances until after you are done
comparison_engine = StructureMatcher(ltol=.2, stol=.5, angle_tol=10, primitive_cell=True, 
                                     scale=True, attempt_supercell=True, 
                                     comparator=FrameworkComparator())
##### Don't edit code above #####

In [33]:
print(comparison_engine.fit(ac_structure, ac_structure))

True


We know that the high-temperature phase of cristobalite, $\beta$-cristobalite [(mp-546794)](https://materialsproject.org/materials/mp-546794/), has a very similar structure to $\alpha$-cristobalite. Let's see if the structure matcher agrees. Please retreive the structure for `mp-546794` and then compare it with our prototype structure. 

_Hint: No need to re-initialize the structure matcher. Just call it's `fit` function again._

In [34]:
# Your code here
with MPRester("<your API key>") as mpr:
    bc_structure = mpr.get_structure_by_material_id("mp-546794")
print(comparison_engine.fit(bc_structure, ac_structure))

True


Just to make sure we haven't increased the tolerances too much, please try it against a random compound to make sure it's not matching dissimilar structures.

_Hint: "mp-4991" is a good random MPID you can use_

In [35]:
# Your code here
with MPRester("<your API key>") as mpr:
    bc_structure = mpr.get_structure_by_material_id("mp-4991")
print(comparison_engine.fit(bc_structure, ac_structure))

False


## Step 3: Get a set of candidate structures to compare

Now that we have our comparator, we need some candidates to screen! 

Imagine that we have an experimental colleague, Soren Tsarpinski, who is an expert at synthesizing vanadate compounds. We have a hunch that some of the vanadates coming out of Dr. Tsarpinski's lab might have similar structures to $\alpha$-cristobalite and therefore might have negative Poisson's ratios. Let's see if we're right:

For our search, we want to start with a set of structures that are:
* Computationally tractable, so not too many sites (i.e `nsites = (0, 50)` is a good range)
* Not too unlikely to be synthesizable (energy above hull <=100 meV/atom, i.e. `energy_above_hull = (0.0, 0.1)`)
* Have a "vanadate" composition, i.e. `"*V3O8"`

Construct and execute a query to get the `structure`, `material_id`, and `formula_pretty` for all materials that match these criteria:

In [42]:
# Your code here
with MPRester("<your API key>") as mpr:
    
    vanadate_ids = [entry.material_id for entry in mpr.query(chemsys_formula="*V3O8", fields=["material_id"])]
    
    vanadates = mpr.query(material_ids=vanadate_ids, 
                          energy_above_hull=(0.0,0.1),
                          nsites=(0,50),
                          fields=["structure", "material_id", "formula_pretty"])

## Step 4: Screen the vanadates for similar structures to $\alpha$-cristobalite and then check if our hypothesis is valid by querying for their Poisson's ratios. 

Now that we have a list of vanadates, let's screen it for similar structures. After we have the similar structures, make one final query to the Materials API to retrieve the `formula_pretty` and `homogeneous_possion` property for each one. 

_Hint: Create an empty list for matches and then iterate through the vanadate entries with a for loop. If its structure is similar to $\alpha$-cristobalite, append its `material_id` to a list of matches. After you have a match list, query the Materials API for entries with a `material_id` that is in your matches._

In [49]:
# Your code here
matches = []
for entry in vanadates:
    if comparison_engine.fit(ac_structure, entry.structure):
        matches.append(entry.material_id)

with MPRester("<your API key>") as mpr:
    elastic_data = mpr.query(material_ids=matches, fields=['material_id', 'formula_pretty', 'homogeneous_poisson'])

for e in elastic_data:
    print(e.material_id, e.formula_pretty, e.homogeneous_poisson)

mp-753738 V3CoO8 None
mp-774945 MnV3O8 None
mp-775001 V3FeO8 -0.06158139322390399
